Use this to check if your setup is good for backreaction.

In [ ]:
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles

using SpheriCo

In [ ]:
# number of points in radial grid
D = 3
Nr = (128)*2^D + 1 + 2 # +1 for last point, + 2 for ghosts left of r=0
r_max = 200
# grid spacing
hr = r_max / (Nr-3)
r  = range(-2*hr, r_max, length=Nr) 
println(r)
println("hr = ",hr)

Classical initial data

In [ ]:
Phi_ID = zeros(length(r))
Psi_ID = zeros(length(r))

# amplitude
a = 1.0
# position of center
b = 0.0
#width
c = 40.0
for i in 1:length(r)
    Phi_ID[i] = SpheriCo.classical.Φ_ID(r[i], a, c, b)
    Psi_ID[i] = SpheriCo.classical.Ψ_ID(r[i], a, c, b)
end


check what is tha characteristic length (λ_char) of the classical initial data (when the amplitude drops to 5% of its max). This afects what is a good mPV, which needs to e much bigger than 2π/λ_char.

In [ ]:
i = 3
while Phi_ID[i] > maximum(Phi_ID)*0.05
    i+=1
end
println("λ_char = ", r[i])
println("2π/λ_char = ", 2π/r[i])

Plot ID for Φ

In [ ]:
ri_min = 3
ri_max = Nr

plot(r[ri_min:ri_max], Phi_ID[ri_min:ri_max], linewidth = 2, frame = true, wsize = (800,400),
     xlim=(0.0,r_max))


Check if the mPV choice is good. You also want dt<< 1/omega_max

In [ ]:
mPV = 1.0
mlist = [0.0, 1.0*mPV, sqrt(3.0)*mPV, 2.0*mPV]
println(mlist)
dk = π/r_max
lmax = 30
kmax = 10
println("dk = ", dk)
omega_max = sqrt(kmax*dk + 2*mPV)
println("1/omega_max = ", 1/omega_max)
println("dt = hr/16 = ", hr/16)

Build the ID for the quantum modes

In [ ]:
v_quantum = zeros(ComplexF64, (length(r), Int(lmax+1), Int(kmax), length(mlist), 3));
println("size(v_quantum) = ", size(v_quantum))

for ll in 0:lmax
    for kk in 1:kmax
        for mm in 1:length(mlist)
            # index for uq, ψq, and πq
            uqi = 1
            ψqi = 2
            πqi = 3
            mi = mlist[mm]
            for i in 3:Nr
                k=kk*dk
                # uq; even
                v_quantum[i, Int(ll+1), Int(kk), mm, uqi] = SpheriCo.quantum.uq_ID(r[i], k, mi, Float64(ll) )
                # ψq; odd
                v_quantum[i, Int(ll+1), Int(kk), mm, ψqi] = SpheriCo.quantum.ψq_ID(r[i], k, mi, Float64(ll) )
                # πq; even
                v_quantum[i, Int(ll+1), Int(kk), mm, πqi] = SpheriCo.quantum.πq_ID(r[i], k, mi, Float64(ll) ) 
            end
            # uq ghosts; even
            v_quantum[:, Int(ll+1), Int(kk), mm, uqi] = SpheriCo.even_ghosts(v_quantum[:, Int(ll+1), Int(kk), mm, uqi])
            # ψq ghosts; odd
            v_quantum[:, Int(ll+1), Int(kk), mm, ψqi] = SpheriCo.odd_ghosts(v_quantum[:, Int(ll+1), Int(kk), mm, ψqi])
            # πq ghosts; even
            v_quantum[:, Int(ll+1), Int(kk), mm, πqi] = SpheriCo.even_ghosts(v_quantum[:, Int(ll+1), Int(kk), mm, πqi]) 
        end 
    end
end

Check if they are resolved well in this grid.

In [ ]:
ri_min = 3
ri_max = Nr
println("hr = ", hr)

ll = 1.0
kk = 10.0
f1 = real.((v_quantum)[:,Int(ll)+1,Int(kk),4,2]) 
plot(r[ri_min:ri_max], f1[ri_min:ri_max],
     label = "f1", linewidth = 2, frame = true, wsize = (800,400),
     xlim=(0,r_max))
